In [2]:
## Scrapping Behance

In [1]:
#Libraries
import pandas as pd
import re

from getpass import getpass
import random
import time

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options

In [2]:
## first we need to initiate the driver##
options = Options()
options.page_load_strategy = 'normal'
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

[WDM] - 

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Get LATEST driver version for 94.0.4606
[WDM] - Trying to download new driver from https://chromedriver.storage.googleapis.com/94.0.4606.61/chromedriver_win32.zip
[WDM] - Driver has been saved in cache [C:\Users\catar\.wdm\drivers\chromedriver\win32\94.0.4606.61]


In [3]:
driver.maximize_window() ## maximize our page

In [4]:
# open the website
driver.get('https://www.behance.net/galleries?tracking_source=nav20')
## settle as main window
main_window = driver.current_window_handle

In [5]:
# Login in
signin = driver.find_elements_by_css_selector("button[class^='Btn-button-BGn Btn-base-M-O Btn-normal-hI4 js-adobeid-signin PrimaryNav-a11yButton-2Cl e2e-PrimaryNav-Signin']")[0].click()
time.sleep(2.5) ## sleep timer in order to mimic human interaction

In [6]:
# Insert email
email = getpass('Enter your email') ## input email
xpath_page = "//input[@type='email']"
email_box = driver.find_element_by_xpath(xpath_page)
email_box.send_keys(email)  ## send email to webpage
time.sleep(3.5) ## sleep timer in order to mimic human interaction

## click to continue
send_email = driver.find_elements_by_css_selector("button[data-id^='EmailPage-ContinueButton']")[0].click()
time.sleep(2.5) ## sleep timer in order to mimic human interaction

In [7]:
## Insert password
password = getpass('Enter your password') ## input password
xpath_page = "//input[@type='password']"
password_box = driver.find_element_by_xpath(xpath_page)

#password_box = driver.find_elements_by_css_selector("input[type^='password']")  ## find the place to input the password
password_box.send_keys(password)  ## send password to webpage
time.sleep(1.3) ## sleep timer in order to mimic human interaction

## click to continue
send_pass = driver.find_elements_by_css_selector("button[aria-label^='Continuar']")[0].click()
time.sleep(2.5) ## sleep timer in order to mimic human interaction

In [14]:
## ignore and click to continue
## only run if the login without password page appear
#ignore = driver.find_elements_by_css_selector("button[data-id^='PasswordlessOptInPP-continue-button']")[0].click()
#time.sleep(2.5) ## sleep timer in order to mimic human interaction

In [8]:
## acept cookies and click to continue
cookies = driver.find_elements_by_css_selector("button[id^='onetrust-accept-btn-handler']")[0].click()
time.sleep(2.5) ## sleep timer in order to mimic human interaction

In [64]:
## Now lets see each project information

In [9]:
## First lets get our project links

def get_links(results_number):
    ## first refresh the page
    driver.refresh()
    ## scroll to load page in order to have the number of results we want
    while True:
        xpath_page = "//li[@class='ContentGrid-gridItem-2Ad e2e-ContentGrid-item']"
        page_scroll = driver.find_elements_by_xpath(xpath_page)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        if len([element.get_attribute("style") for element in page_scroll]) > int(results_number):
            break
        time.sleep(random.uniform(3,5))  ## sleep timer in order to mimic human interaction
    
    ## get links from the results
    project_links = [element.get_attribute("href") for element in driver.find_elements_by_xpath(xpath_page+"//a[@class='ProjectCoverNeue-coverLink-2Hl js-project-cover-image-link js-project-link e2e-ProjectCoverNeue-link']")]

    ## get project photo
    project_photos = [element.get_attribute("src") for element in driver.find_elements_by_xpath(xpath_page+"//img[@class='ProjectCoverNeue-image-13g js-cover-image']")]

    ## dataframe with this info
    projects_data = pd.DataFrame([project_links,project_photos]).T
    projects_data = projects_data.dropna().drop_duplicates()
    projects_data.columns = ['project_links', 'project_photos']
    return projects_data

In [13]:
test = get_links(30)

In [ ]:
## Now lets scrap

In [13]:
## lets open the project in a new tab

# open new blank tab
driver.execute_script("window.open();")

# switch to the new window which is second in window_handles array
driver.switch_to_window(driver.window_handles[1])

# open successfully and close
driver.get(test['project_links'].iloc[3])

<ipython-input-13-543be129291a>:7: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(driver.window_handles[1])


In [36]:
## after get the information we need to close the page and return to the main one
driver.close()

# back to the main window
driver.switch_to_window(main_window)

<ipython-input-36-269aee147c1b>:5: DeprecationWarning: use driver.switch_to.window instead
  driver.switch_to_window(main_window)


In [11]:
## Once we have a list with unique results links lets scrapp each one

def get_info(projects):
    
    # Create to add the info scrapped
    projects_name = []
    projects_likes = []
    projects_visualizations = []
    projects_nr_comments = []
    projects_dates = []
    projects_tools = []
    projects_creative = []
    projects_tags = []
    artists_links = []
    total_photos_links = []
    total_photos_number = []
    total_commments = []

    ## lets open the project in a new tab
    for project in projects['project_links']:
        ## to keep track of our position
        print("%d _ %d" % (len(projects_name), len(projects['project_links'])))

        ## open new blank tab
        driver.execute_script("window.open();")

        ## switch to the new window which is second in window_handles array
        driver.switch_to_window(driver.window_handles[1])

        ## open successfully 
        driver.get(project)

        ## get our project info
        xpath_page = "//div[@class='Project-projectInfoSection-3it e2e-Project-infoSection']"

        ## get project name
        project_name = driver.find_element_by_xpath(xpath_page+"//div[@class='Project-projectName-2sV']").text
        projects_name.append(project_name)
        time.sleep(random.uniform(1,2))

        ## get likes
        project_likes = driver.find_element_by_xpath(xpath_page+"//div[@class='Project-projectStat-6Y3 beicons-pre beicons-pre-thumb']").text
        projects_likes.append(project_likes)
        time.sleep(random.uniform(1,2))

        ## get visualizations
        project_visualizations = driver.find_element_by_xpath(xpath_page+"//div[@class='Project-projectStat-6Y3 beicons-pre beicons-pre-eye']").text
        projects_visualizations.append(project_visualizations)
        time.sleep(random.uniform(1,2))

        ## get comments
        project_nr_comments = driver.find_element_by_xpath(xpath_page+"//div[@class='Project-projectStat-6Y3 beicons-pre beicons-pre-comment']").text
        projects_nr_comments.append(project_nr_comments)
        time.sleep(random.uniform(1,2))

        ## get publication date
        project_date = driver.find_element_by_xpath(xpath_page+"//div[@class='Project-projectPublished-1UI']//time").text
        projects_dates.append(project_date)
        time.sleep(random.uniform(1,2))

        ## get tools
        project_tools = [element.get_attribute("alt") for element in driver.find_elements_by_xpath("//div[@class='ProjectTools-section-1cb e2e-Project-Tools-tools']//img[@class='ProjectTools-toolIcon-IoN']")]
        projects_tools.append(project_tools)
        time.sleep(random.uniform(1,2))

        ## get creative fields
        project_creative = [element.get_attribute("alt") for element in driver.find_elements_by_xpath("//li[@class='ProjectTools-creativeField-3n3']//img[@class='PillShapedInfoButton-background-10c']")]
        projects_creative.append(project_creative)
        time.sleep(random.uniform(1,2))

        ## get project tags
        project_tags = [element.text for element in driver.find_elements_by_xpath("//ul[@class='ProjectTags-projectTags-1IC ProjectTags-usePillShapedTags-1XX js-project-tags ProjectInfo-infoBlocks-2hM ProjectInfo-projectTags-2fx']//li//a")]
        projects_tags.append(project_tags)
        time.sleep(random.uniform(1,2))

        ## artist name / location / avaliações / exibições / seguidores - get artist page
        artist_links = [element.get_attribute("href") for element in driver.find_elements_by_xpath("//a[@class='ProjectOwnersInfo-userName-2oz js-mini-profile qa-user-link e2e-ProjectOwnersInfo-user-link']")]
        artists_links.append(artist_links)
        time.sleep(random.uniform(1,2))

        ## elements in project
        photos_links = [element.get_attribute("src") for element in driver.find_elements_by_xpath("//div[@class='js-project-module js-project-module--image project-module module image e2e-Project-modules-image project-module-image image-full project-module-image-full Image-root-1tO']//img")]
        total_photos_links.append(photos_links)
        time.sleep(random.uniform(1,2))
        
        ## Number of elements in project
        photos_number = len(photos_links)
        total_photos_number.append(photos_number)
        time.sleep(random.uniform(1,2))

        ## get comments
        while True:  ## open all comments
            try:
                more_comments = driver.find_elements_by_css_selector("div[class='js-see-more comments-pagination']")[0].click()
                time.sleep(random.uniform(1,2))  ## sleep timer in order to mimic human interaction
            except:
                break

        commments = [element.text for element in driver.find_elements_by_xpath("//div[@class='comment-text js-comment-text']")]
        total_commments.append(commments)
        time.sleep(random.uniform(1,2))

        ## after get the information we need to close the page
        driver.close()

        # return to the main window
        driver.switch_to_window(main_window)


    #lets retrieve dataframe with this information
    projects_data = pd.DataFrame([projects_name,projects_likes,projects_visualizations,projects_nr_comments,projects_dates,projects_tools,projects_creative,projects_tags,artists_links,total_photos_links,total_photos_number,total_commments]).T
    projects_data.columns = ['project_name','nr_likes','nr_visualizations','nr_comments','publication_date','tools_used','creative_fields','tags','artists_profile','project_photos','project_nr_photos','comments']
    projects_data['project_links'] = projects['project_links']
    projects_data['project_main_photo'] = projects['project_photos']

    return projects_data

In [12]:
links = get_links(2500)

In [13]:
links.to_csv("links.csv",sep = ',')

In [38]:
#behance_data = get_info()

In [ ]:
behance_data

In [ ]:
behance_data.to_csv("behance_data.csv",sep = ',')

In [ ]:
## get artist name
artists_name = [element.text for element in driver.find_elements_by_xpath("//div[@class='ProjectOwnersInfo-userInfo-2WK']//a[@class='ProjectOwnersInfo-userName-2oz js-mini-profile qa-user-link e2e-ProjectOwnersInfo-user-link']")]

## get location name
artists_locations = [element.text for element in driver.find_elements_by_xpath("//div[@class='ProjectOwnersInfo-userInfo-2WK']//a[@class='ProjectOwnersInfo-userLocation-_rE beicons-pre beicons-pre-location']")]

pr